In [57]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch as t
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch.nn.functional as F


In [58]:
ds = load_dataset("myvision/gender-classification")

In [59]:
transform = transforms.Compose([
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

def preprocess(example):
    example["image"] = [transform(img) for img in example["image"]]
    return example

ds = ds.with_transform(preprocess)


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1000
    })
    eval: Dataset({
        features: ['image', 'label'],
        num_rows: 1000
    })
})

In [ ]:
train_loader = DataLoader(ds["train"], batch_size=32, shuffle=True)
test_loader  = DataLoader(ds["test"], batch_size=32)
seen=set()
for batch in train_loader:
    X, y = batch["image"], batch["label"]
    seen.add((X.shape, y.shape))
seen


{(torch.Size([8, 3, 224, 224]), torch.Size([8])),
 (torch.Size([32, 3, 224, 224]), torch.Size([32]))}

In [62]:
import torch
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(256 * 5 * 5, 4096),  
            nn.ReLU(),

            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  
        x = self.classifier(x)
        return x


In [63]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

model = AlexNet().to(device)
createrion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),0.0001)

Using device: mps


In [64]:
for epoch in range(5):
    running_loss = 0.0
    
    for batch in train_loader:
        img, label = batch["image"].to(device), batch["label"].to(device)
        
        ypred = model(img)
        loss = createrion(ypred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{5}], Loss: {running_loss / len(train_loader):.4f}")

Epoch [1/5], Loss: 0.5425
Epoch [2/5], Loss: 0.2892
Epoch [3/5], Loss: 0.2236
Epoch [4/5], Loss: 0.1719
Epoch [5/5], Loss: 0.1506


In [65]:
print(f"Epoch [{epoch+1}/{5}], Loss: {running_loss / len(train_loader):.4f}")


Epoch [5/5], Loss: 0.1506


In [66]:

correct = 0
total = 0
with t.no_grad():
    for batch in test_loader:
        
        images, labels = batch["image"].to(device), batch["label"].to(device)
        
        outputs = model(images)
        _, predicted = t.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")
    

Test Accuracy: 92.10%


In [67]:
num_classes = 2
class_correct = [0] * num_classes
class_total = [0] * num_classes

model.eval()  
with t.no_grad():
    for batch in test_loader:
        img, label = batch["image"].to(device), batch["label"].to(device)
        outputs = model(img)
        _, predicted = t.max(outputs, 1)
        
        c = (predicted == label)
        for i in range(len(label)):
            lbl = label[i].item()
            class_correct[lbl] += c[i].item()
            class_total[lbl] += 1

for i in range(num_classes):
    acc = 100 * class_correct[i] / class_total[i]
    print(f"Class {i} Accuracy: {acc:.2f}%")


Class 0 Accuracy: 92.20%
Class 1 Accuracy: 91.40%


In [68]:
import cv2 as cv

In [69]:
import cv2 as cv

img_path = "/Users/bhara-zstch1566/CNN/Project/AlexNet/Geneder Classification/datasset/swathi.png"
img = cv.imread(img_path)  
img = cv.resize(img, (224, 224))  


In [70]:
img.shape

(224, 224, 3)

In [71]:
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)


In [72]:
img_tensor = transform(img).unsqueeze(0)
print(img_tensor.shape)

torch.Size([1, 3, 224, 224])


In [73]:
img_tensor = img_tensor.to(device)
model.eval()
with torch.no_grad():
    outputs = model(img_tensor)
    _, predicted = torch.max(outputs, 1)
    print("Predicted class:", "Male" if predicted.item() else "Female" )


Predicted class: Male


In [74]:
a = train_loader 

In [75]:
b= train_loader

In [76]:
a==b

True

In [77]:
t.save(model.state_dict(), "model.pth")